# Take our .csv files and making them .sql files

In [31]:
#Note: I have been working on this project for a week and went to Office Hours many times. Some things 
#were simply not working and I really tried my best. I understand the logic 
#and what each line was doing but was running into small errors and I was too late the ask the TA or I was unspecific..
#I hope that you can keep that in mind. 

import os
import pandas as pd

import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine

import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine

import pandas as pd
import matplotlib.pyplot as plt

print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 1.4.39
Running PyMongo Version: 4.6.3


# Bring in CSV data to MongoDB

In [32]:
data_dir = os.path.join(os.getcwd(), 'underwear')
data_file = os.path.join(data_dir, 'customers.csv') 
df = pd.read_csv(data_file, header=0, index_col=0, encoding = 'utf-16')
df.head()

,CustomerName,Region,Country,PriceCategory,CustomerClass,LeadSource,Discontinued
CustomerID,,,,,,,
1,C1,Moscow,Russian Federation,1,Large-Scale Wholesaler-1,Referral by the Central Office,0
2,C2,Moscow,Russian Federation,1,Large-Scale Wholesaler-1,Referral by the Central Office,0
3,C3,Moscow,Russian Federation,1,Large-Scale Wholesaler-1,Referral by the Central Office,1
4,C4,Moscow,Russian Federation,4,Large-Scale Wholesaler-1,Referral by the Central Office,1
5,C5,Moscow,Russian Federation,1,Large-Scale Wholesaler-1,Referral by the Central Office,0


In [33]:
data_dir = os.path.join(os.getcwd(), 'underwear')
data_other_file = os.path.join(data_dir, 'order_details.csv')
df_other = pd.read_csv(data_other_file, header=0, index_col=0)
df_other.head()

,OrderID,ProductID,QuantitySold,UnitSalesPrice
OrderDetailID,,,,
2,2,955,5,7.5
3,2,958,5,8.5
5,2,959,5,8.5
6,4,72,50,3.0
7,4,9,50,3.2


# Make connections for SQL and MongoDB to work with Jupyter

In [9]:

    uid : "root"
    pwd : "Passw0rd123"
    hostname : "localhost"
    dbname : "underwear"
    dst_dbname : "underwear2"


# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    "user_name" : "",
    "password" : "password",
    "cluster_name" : "cluster_underwear",
    "cluster_subnet" : "123456",
    "cluster_location" : "local", # "local"
    "db_name" : "underwear_data"
}

In [10]:
def get_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe
    

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()
    

# Create a new Data Warehouse and Connect to It

In [11]:
#create new database and switch connection context to use it 
conn_str = f"mysql+pymysql://{uid}:{pwd}@{hostname}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
connection.execute(f"CREATE DATABASE `{dst_dbname}`;")
connection.execute(f"USE {dst_dbname};")

connection.close()

NameError: name 'uid' is not defined

# Bring in other files

In [12]:
client = get_mongo_client(**mongodb_args) #might need to change to json-> make sure to take a look at that

# Gets the path of the Current Working Directory for this Notebook,
# and then Appends the 'underwear' directory.
data_dir = os.path.join(os.getcwd(), 'underwear')
csv_files = {"order_details":"order_details.csv",
             "customers" : "customers.csv"
             }

set_mongo_collections(client, mongodb_args["db_name"], data_dir, csv_files)   

JSONDecodeError: Extra data: line 1 column 16 (char 15)

# Extract data and put it into a dataframe

In [13]:
#extract data from Source MongoDB Collections into DataFrames
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "customers"

df_customers = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_customers.head(2)

KeyError: "['_id'] not found in axis"

In [ ]:
#extract data from Source MongoDB Collections into DataFrames
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "order_details"

df_order_details = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
df_order_details.head(2)

In [ ]:
#Get Data from Source Database Tables
sql_dim_products = "SELECT * FROM underwear.dim_products;"
df_products = get_dataframe(user_id, pwd, host_name, src_dbname, sql_dim_products)
df_products.head(2)

In [15]:
#Get Data from Source Database Tables
sql_orders = "SELECT * FROM underwear.orders;"
df_orders = get_dataframe(uid, pwd, host_name, src_dbname, sql_orders)
df_orders.head(2)

NameError: name 'uid' is not defined

In [16]:
#Get Data from Source Database Tables
sql_dim_date = "SELECT * FROM underwear.dim_date;"
df_date = get_dataframe(uid, pwd, host_name, src_dbname, sql_dim_date)
df_date.head(2)

NameError: name 'uid' is not defined

# Peform Transformations on the dim tables

In [22]:
#Perform Transformations 
#For Customers
# 1. Create a List that enumerates the names of each column you wish to remove (drop) from the Pandas DataFrame
drop_cols = ['Region','Country','CustomerClass','LeadSource']
df_customers.drop(drop_cols, axis=1, inplace=True)

# 3. Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_customers.insert(0, "customer_key", range(1, df_customers.shape[0]+1))

# 4. Display the first 2 rows of the dataframe to validate your work
df_customers.head(2)

NameError: name 'df_customers' is not defined

In [ ]:
#For Products
# 1. Create a List that enumerates the names of each column you wish to remove (drop) from the Pandas DataFrame
drop_cols = ['ModelDescription','FabricDescription','ProductLine','PackSize']
df_products.drop(drop_cols, axis=1, inplace=True)

# 3. Insert a new column, with an ever-incrementing numeric value, to serve as the primary key.
df_products.insert(0, "product_key", range(1, df_products.shape[0]+1))

# 4. Display the first 2 rows of the dataframe to validate your work
df_products.head(2)

# Start the fact_orders table creation: get data from the dim_date table

In [21]:
#Get the Data from the Date Dimension Table -
sql_dim_date = "SELECT date_key, full_date FROM underwear2.dim_date;"
df_dim_date = get_dataframe(sql_dim_date, **mysql_args)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(2)

OperationalError: (pymysql.err.OperationalError) (1049, "Unknown database 'underwear2'")
[SQL: SELECT date_key, full_date FROM underwear2.dim_date;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

# Merge dim_products and dim_date based off of InventoryDate

In [23]:
df_dim_inventory_date = df_dim_date.rename(columns={"date_key" : "inventory_date_key", "full_date" : "InventoryDate"})
df_dim_inventory_date.InventoryDate = df_products.InventoryDate.astype('datetime64[ns]').dt.date
df_products = pd.merge(df_products, df_dim_inventory_date, on='InventoryDate', how='left')
df_products.drop(['InventoryDate'], axis=1, inplace=True)
df_products.head(2)

NameError: name 'df_dim_date' is not defined

# Take your new transformed dataframes and put them in the new Warehouse

In [24]:
#Load Transformed Dataframes into new Data Warehouse
db_operation = "insert"

tables = [('dim_customers', df_customers, 'customer_key'),
          ('dim_products', df_products, 'product_key')]

NameError: name 'df_customers' is not defined

In [25]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

NameError: name 'tables' is not defined

# Correspond the dim_date w/ Order table

In [26]:
# Lookup the Surrogate Primary Key (date_key) that Corresponds to the "OrderDate" Column. - potentially not needed
df_dim_order_date = df_dim_date.rename(columns={"date_key" : "order_date_key", "full_date" : "OrderDate"})
df_dim_order_date.OrderDate = df_fact_pos.OrderDate.astype('datetime64[ns]').dt.date
df_orders = pd.merge(df_fact_pos, df_dim_order_date, on='OrderDate', how='left')
df_orders.drop(['OrderDate'], axis=1, inplace=True)
df_orders.head(2)

NameError: name 'df_dim_date' is not defined

In [27]:
# Lookup the Surrogate Primary Key (date_key) that Corresponds to the "ShipDate" Column. - potentially not needed
df_dim_ship_date = df_dim_date.rename(columns={"date_key" : "ship_date_key", "full_date" : "ShipDate"})
df_dim_ship_date.ShipDate = df_fact_pos.submitted_date.astype('datetime64[ns]').dt.date
df_orders = pd.merge(df_fact_pos, df_dim_ship_date, on='ShipDate', how='left')
df_orders.drop(['ShipDate'], axis=1, inplace=True)
df_orders.head(2)

NameError: name 'df_dim_date' is not defined

# Create fact_order table

In [28]:
#create fact orders
df_fact_orders = pd.merge(df_orders, df_other, on = 'OrderId', how = 'right')
df_fact_orders.head(2)

NameError: name 'df_orders' is not defined

In [29]:
df_fact_orders.shape

NameError: name 'df_fact_orders' is not defined

# Connect the dim tables to the fact_order table

In [30]:
# Extract the 'primary key' and the 'business key' from the "dim_products" dimension table
sql_dim_products = "SELECT product_key, product_id FROM underwear2.dim_products;"
df_dim_products = get_dataframe(sql_dim_products, **mysql_args)
df_dim_products.head(2)

OperationalError: (pymysql.err.OperationalError) (1049, "Unknown database 'underwear2'")
[SQL: SELECT product_key, product_id FROM underwear2.dim_products;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
# Extract the 'primary key' and the 'business key' from the "dim_customers" dimension table
sql_dim_customers = "SELECT customer_key, customer_id FROM underwear2.dim_customers;"
df_dim_customers = get_dataframe(sql_dim_customers, **mysql_args)
df_dim_customers.head(2)

# Merge the dimension tables and the fact tables

In [ ]:
# Merge the "fact_inventory" and "dim_products" dataframes on the 'product_id' to
# get the 'product_key'. Then drop the 'product_id' column and display the results.
df_fact_products = pd.merge(df_fact_products, df_dim_products, on = 'ProductId', how = 'inner')
df_fact_products.drop(['ProductId'], axis = 1, inplace = True)
df_fact_products.head(2)

In [ ]:
# Merge the "fact_inventory" and "dim_customers" dataframes on the 'CustomerId' to
# get the 'customer_key'. Then drop the 'CustomerId' column and display the results.
df_fact_customers = pd.merge(df_fact_customers, df_dim_customers, on = 'CustomerId', how = 'inner')
df_fact_customers.drop(['CustomerId'], axis = 1, inplace = True)
df_fact_customers.head(2)

# Put the fact tables back into the new data warehouse

In [ ]:
#Inserting fact table back into SQL data warehouse
dataframe = df_fact_customers
table_name = 'fact_customers'
primary_key = 'fact_customers_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

In [ ]:
#Inserting fact table back into SQL data warehouse
dataframe = df_fact_products
table_name = 'fact_products'
primary_key = 'fact_products_key'
db_operation = "insert"

set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)

# SQL Queries to get the total and average purchase prices

In [ ]:
#SQL Query
sql_fact_products = """

SUM(products.PurchasePrice) AS 'TotalPurchasePrice'
FROM underwear2.fact_products AS po
INNER JOIN underwear2.dim_products AS s
ON po.ProductID = s.ProductID
GROUP BY s.ProductID
"""

In [ ]:
#SQL Query
sql_fact_products = """

AVG(products.PurchasePrice) AS 'AveragePurchasePrice'
FROM underwear2.fact_products AS po
INNER JOIN underwear2.dim_products AS s
ON po.ProductID = s.ProductID
GROUP BY s.ProductID
"""